In [10]:
import pandas as pd
from scripts.preprocessing import preprocess_data

# Load dataset
df = pd.read_csv("data/creditcard.csv")

# Preprocess
X_train, X_test, y_train, y_test = preprocess_data(df)

ModuleNotFoundError: No module named 'scripts.preprocessing'